##### Optimal Stake = (T/SIP) * IIP  
##### Profit = (individual stake) * Odds - (other stakes)

Where,  
T -> Budget  
SIP -> Sum of Implied Probability  
IIP -> Individual Implied Probability

In [5]:
import requests
import json
import pandas as pd

In [14]:
# Stake for investment
total_stake = 1000
return_threshold = 0.5 # in percent

In [15]:
# Defining API and extracting data

api_key = "9e7057b668f6f16479dd96e7373bef5d"
region = "eu,uk"
market = "h2h"
sport_key = "soccer_epl"
url = f'https://api.the-odds-api.com/v4/sports/{sport_key}/odds/?apiKey={api_key}&regions={region}&markets={market}'

odds_response = requests.get(url)
odds_data = json.loads(odds_response.text)

In [16]:
# Dataset Construction

rows_list = []
for game in odds_data:
    for bookmaker in game['bookmakers']:
        for market_ in bookmaker['markets']:
            for outcome in market_['outcomes']:
                row = {
                    'game_id': game['id'],
                    'sport_key': game['sport_key'],
                    'sport_title': game['sport_title'],
                    'home_team': game['home_team'],
                    'away_team': game['away_team'],
                    'commence_time': game['commence_time'],
                    'bookmaker_key': bookmaker['key'],
                    'bookmaker_title': bookmaker['title'],
                    'bookmaker_last_update': bookmaker['last_update'],
                    'market_key': market_['key'],
                    'market_last_update': market_['last_update'],
                    'outcome_name': outcome['name'],
                    'outcome_price': outcome['price']
                }
                rows_list.append(row)
df = pd.DataFrame(rows_list)

# Data Cleaning

df = df[~df['bookmaker_key'].isin(['betfair_ex_uk', 'betfair_ex_eu', 'matchbook'])]
idx = df.groupby(['game_id', 'outcome_name'])['outcome_price'].idxmax()
df_arbitrage = df.loc[idx].copy()

# Calculations

df_arbitrage['implied_prob'] = 1/df['outcome_price']
df_arbitrage['sum_implied'] = df_arbitrage.groupby('game_id')['implied_prob'].transform('sum')
df_arbitrage = df_arbitrage[df_arbitrage['sum_implied']<1]
df_arbitrage['stake'] = (total_stake/df_arbitrage['sum_implied'])*df_arbitrage['implied_prob']
df_arbitrage['ror'] = (1 - df_arbitrage['sum_implied'])*100

In [20]:
df_arbitrage[df_arbitrage['ror'] > return_threshold].loc[:,['home_team', 'away_team', 'bookmaker_title', 'outcome_name', 'outcome_price', 'stake', 'ror']]

,home_team,away_team,bookmaker_title,outcome_name,outcome_price,stake,ror
1386,Liverpool,Chelsea,Suprabets,Chelsea,4.20,239.726148,0.680322
1382,Liverpool,Chelsea,1xBet,Draw,4.22,238.590005,0.680322
1387,Liverpool,Chelsea,Suprabets,Liverpool,1.93,521.683846,0.680322
632,West Ham United,Ipswich Town,Suprabets,Draw,4.20,241.133170,1.259856
687,West Ham United,Ipswich Town,Unibet,Ipswich Town,4.60,220.165068,1.259856
631,West Ham United,Ipswich Town,Suprabets,West Ham United,1.88,538.701762,1.259856
1419,Nottingham Forest,Crystal Palace,Suprabets,Crystal Palace,2.91,345.771083,0.615572
1421,Nottingham Forest,Crystal Palace,Suprabets,Draw,3.83,262.713799,0.615572
1417,Nottingham Forest,Crystal Palace,1xBet,Nottingham Forest,2.57,391.515117,0.615572
728,Everton,Newcastle United,1xBet,Draw,3.87,259.779550,0.531842
